## Preuzimanje podataka sa portala Sunny portal

In [1]:
#%pip install requests
#%pip install bs4
#%%pip install lxml

In [6]:
import pandas as pd
import datetime as dt
import numpy as np
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import time
import re
from selenium.webdriver import ActionChains



Za preuzimanje podataka sa web stranice, prvo je potrebno odrediti godine za koje će se preuzeti, te formirati rang datuma od početka pa do kraja godine u formatu M/D/YYYY jer se takav format pojavljuje u linkovima na web-stranici. Za ovo služi funkcija **define_year**.

In [3]:
def define_year(year,number_of_days1):
    start_date = dt.date(year, 1 , 1)
    number_of_days = number_of_days1
    date_list = [((start_date + dt.timedelta(days = day)).strftime("%#m/%#d/%Y")) for day in range(number_of_days)]
    #date_list2=[((start_date + dt.timedelta(days = day)).strftime("%Y-%m-%d")) for day in range(number_of_days)]
    return date_list

Podaci će se ekstraktovati na način da se link web-stranice mijenja u zavisnosti od datuma koji je postavljen u date_list_1, te će se u HTML fajlu (page source) naći tabela sa podacima proizvodnje električne energije za taj datum. Za ovo je potrebno imati link koji odgovara svakoj solarnoj elektrani pojedinačno. Za jednu solarnu elektranu su svi dijelovi linka isti izuzev datuma koji se mijenja. Prema tome, kada se napravi petlja kojom se mijenja datum i ubacuje u ostatak linka, na HTML stranici se pronalazi odgovarajuća tabela sa podacima i spašava za dalje procesiranje. Ovaj dio se odrađuje putem funkcije **extract_data** koja kao ulazne podatke prima datume u jednoj godini, te dva dijela linka izmđeu kojih će se taj datum ubaciti.

In [4]:
def extract_data(date_list_1,link1,link2):
    list_csvs=[None]*len(date_list_1)

    for k,value in enumerate(date_list_1):
        #link=link1+value+link2
        link=link1+value+link2
        
        res = requests.get(link)

#print(res.text)
#print(res.status_code)

        page = requests.get(link)
        soup = BeautifulSoup(page.content, 'html.parser')
    #page_title = soup.title.text
    #print(page_title)

        table1 = soup.find('table', id='ctl00_ContentPlaceHolder1_UserControlChartValues1_Table1')
    	#table1

 
        csvContent = ''
        for j in table1.find_all('tr'):
            row_data = j.find_all('td')

            for cell in row_data:
                csvContent = csvContent + cell.text + ','
            csvContent= csvContent[:-1]
            csvContent = csvContent + '\n'
    
        #print(csvContent,k)
        list_csvs[k]=csvContent
    return list_csvs
    

Prikupljeni podaci se nalaze u tekstualnom obliku, te ih je potrebno pretvoriti u tabelarni prikaz kako bi se mogli spasiti u .xls formatu. list_csvs je lista tekstualnih podataka za jednu čitavu godinu (ima 365, odnosno 366 elemenata za jednu godinu). Ti elementi se najprije pretvaraju u tabele putem funkcije **write_to_dataframe**, a zatim se te tabele spajaju u jednu putem funkcije **concat_and_save**.

In [5]:
def write_to_dataframe(list_csvs):
    dfs=[None]*len(list_csvs)
    for k,value in enumerate(list_csvs):
        with open('output.csv','w',newline='') as f:
            f.write(list_csvs[k])
        dfs[k]=pd.read_csv("output.csv")
    return dfs

In [6]:
def concat_and_save(dfs,name):
    df_tot=pd.concat(dfs,axis=0)
    df_tot.to_excel(name)
    return df_tot

In [8]:
#enelectronic
link_ene1="https://www.sunnyportal.com/Templates/PublicChartValues.aspx?ID=6e2898bd-52c5-455a-81f8-1c5c492aa03f&endTime="
link_ene2="%2011:59:59%20PM&splang=en-US&plantTimezoneBias=60&name="
#krenica i hodovo se nalaze u istom fajlu
link_kr_hod1="https://www.sunnyportal.com/Templates/PublicChartValues.aspx?ID=0ada0c8e-6bcd-483b-9798-1bc3ff24b6a7&endTime=" 
link_kr_hod2="%2011:59:59%20PM&splang=en-US&plantTimezoneBias=60&name="

#definiranje godine i opsega datuma od početka do kraja
date_list_ene2019=define_year(2019,365)
date_list_ene2020=define_year(2020,366)
date_list_ene2021=define_year(2021,365)


date_list_kr_hod2019=define_year(2019,365)
date_list_kr_hod2020=define_year(2020,366)
date_list_kr_hod2021=define_year(2021,365)

#izvlačenje podataka  - Enelectronic 2019
list_csvs_ene2019=extract_data(date_list_ene2019,link_ene1,link_ene2)


Način pohrane podataka za Enelectronic je takav da spašava podatke za 48 sati umjesto za 24, te će se radi izbjegavanja dupliciranja datuma preuzeti svaki drugi element liste list_csvs_ene i spasiti u **listu new_list_ene**. To znači da će podaci biti spašeni kao 1.1.2019, 2.1.2019.,3.1.2019 umjesto 1.1.2019,1.1.2019.,2.1.2019.,2.1.2019.  Za FNE Krenica1 i Hodovo1 podaci su spašeni u opsegu za 24 sata, te za njih ovaj korak nije potreban.

In [11]:

new_list_ene_2019=[]

for value in range(0,len(list_csvs_ene2019),2):
    new_list_ene_2019.append(list_csvs_ene2019[value])

In [12]:
#kreiranje tabelarnog prikaza i spašavanje u .xlsx formatu

df_ene_2019=write_to_dataframe(new_list_ene_2019)
df_ene_2019_tot=concat_and_save(df_ene_2019,"enelectronic_2019.xlsx")

In [14]:
#ponavljanje postupka za 2020. i 2021.godinu
list_csvs_ene2020=extract_data(date_list_ene2020,link_ene1,link_ene2)
list_csvs_ene2021=extract_data(date_list_ene2021,link_ene1,link_ene2)

In [21]:
#izvlačenje podataka  - krenica, hodovo za sve 3 godine
list_csvs_kr_hod_2019=extract_data(date_list_kr_hod2019,link_kr_hod1,link_kr_hod2)

In [25]:
list_csvs_kr_hod_2020=extract_data(date_list_kr_hod2020,link_kr_hod1,link_kr_hod2)

In [23]:
list_csvs_kr_hod_2021=extract_data(date_list_kr_hod2021,link_kr_hod1,link_kr_hod2)

In [19]:
#ponavljanje postupka za 2020.2021 godinu - Enelectronic - preuredit će se u funckiju

new_list_ene_2020=[]
new_list_ene_2021=[]
for value in range(0,len(list_csvs_ene2020),2):
    new_list_ene_2020.append(list_csvs_ene2020[value])
    
for value in range(0,len(list_csvs_ene2021),2):
    new_list_ene_2021.append(list_csvs_ene2021[value])

In [20]:
df_ene_2020=write_to_dataframe(new_list_ene_2020)
df_ene_2020_tot=concat_and_save(df_ene_2020,"enelectronic_2020.xlsx")

df_ene_2021=write_to_dataframe(new_list_ene_2021)
df_ene_2021_tot=concat_and_save(df_ene_2021,"enelectronic_2021.xlsx")



In [22]:
df_kr_hod_2019=write_to_dataframe(list_csvs_kr_hod_2019)
df_kr_hod_2019_tot=concat_and_save(df_kr_hod_2019,"kr_hod_2019.xlsx")

In [26]:
df_kr_hod_2020=write_to_dataframe(list_csvs_kr_hod_2020)
df_kr_hod_2020_tot=concat_and_save(df_kr_hod_2020,"kr_hod_2020.xlsx")

In [24]:
df_kr_hod_2021=write_to_dataframe(list_csvs_kr_hod_2021)
df_kr_hod_2021_tot=concat_and_save(df_kr_hod_2021,"kr_hod_2021.xlsx")

Format HTML stranice za SE Energy 1 i SE "Čule -promet" je nešto drugačiji u odnosu na prethodne, te je potrebno modifikovati kod za dobijanje podataka  i koristi se  paket Selenium i Chrome web driver  sa lokacijom pohrane koji se mora downloadovati i instalirati. Stranica u jednom trenutku može postati previše spora za učitavanje iako su postavljene opcije čekanja. U tom slučaju je potrebno pogledati zadnji spašeni podatak df[-1], te pokrenuti novu iteraciju od tog datuma.

In [7]:
def define_year(year,number_of_days1):
    start_date = dt.date(year, 11 , 2)
    number_of_days = number_of_days1
    date_list = [((start_date + dt.timedelta(days = day)).strftime("%#m/%#d/%Y")) for day in range(number_of_days)]
    #date_list2=[((start_date + dt.timedelta(days = day)).strftime("%Y-%m-%d")) for day in range(number_of_days)]
    return date_list

In [8]:
date_list1=define_year(2019,100)

In [9]:
cule_promet="https://www.sunnyportal.com/Templates/PublicPageOverview.aspx?page=7b8e7dd9-4d7a-4539-9501-f2f36df09d61&plant=0b181aa3-03c7-446c-af6d-3ed6457a5394&splang=en-US"

se="https://www.sunnyportal.com/Templates/PublicPageOverview.aspx?page=84cbfaec-29e6-4ec4-adf9-73ae81533e47&plant=c5f9ed02-1f74-43e2-bf16-b8748c97cf10&splang=en-US"

In [ ]:
#cule,se
# Path do google chrom webdriver

PATH = "C:\\Users\\EhlimanaJugo\\Desktop\\chromedriver.exe"
driver = webdriver.Chrome(PATH)

window_before = driver.window_handles[0]
# url stranice koju hocemo scrape-ati
URL_FOR_SCRAPING = se

driver.get(URL_FOR_SCRAPING)
#prihvatanje cookiesa
time.sleep(4)
cookies=driver.find_element_by_xpath('//*[@id="onetrust-accept-btn-handler"]')
if cookies:
    cookies.click()
#ako treba kliknuti na tab day:
#WebDriverWait(driver,4).until(EC.element_to_be_clickable((By.ID,'ctl00_ContentPlaceHolder1_PublicPagePlaceholder_PageUserControl_ctl00_PublicPageLoadFixPage_UserControlShowEnergyAndPower1_LinkButton_TabFront3')))

# nextq=driver.find_element_by_id('ctl00_ContentPlaceHolder1_PublicPagePlaceholder_PageUserControl_ctl00_PublicPageLoadFixPage_UserControlShowEnergyAndPower1_LinkButton_TabFront3').click()
time.sleep(10)
next1 = driver.find_element_by_class_name("basicdatepicker")
driver.execute_script('document.getElementsByClassName("inputText centered cursorPointer")[0].removeAttribute("readonly")')
dfs=[None]*len(date_list1)
list_csvs=[None]*len(date_list1)

for k, value in enumerate(date_list1):
    driver.implicitly_wait(4)
    #lociranje kalendara i isključenje read only
    _ = WebDriverWait(driver, 60).until(EC.presence_of_all_elements_located((By.XPATH, '//*[@id="ctl00_ContentPlaceHolder1_PublicPagePlaceholder_PageUserControl_ctl00_PublicPageLoadFixPage_UserControlShowEnergyAndPower1__datePicker_textBox"]')))
    driver.execute_script('document.getElementsByClassName("inputText centered cursorPointer")[0].removeAttribute("readonly")')
    _ = WebDriverWait(driver, 60).until(EC.presence_of_all_elements_located((By.XPATH, '//*[@id="ctl00_ContentPlaceHolder1_PublicPagePlaceholder_PageUserControl_ctl00_PublicPageLoadFixPage_UserControlShowEnergyAndPower1__datePicker_textBox"]')))
    next4x=driver.find_element_by_xpath('//*[@id="ctl00_ContentPlaceHolder1_PublicPagePlaceholder_PageUserControl_ctl00_PublicPageLoadFixPage_UserControlShowEnergyAndPower1__datePicker_textBox"]')
    next4=driver.find_element_by_xpath('//*[@id="ctl00_ContentPlaceHolder1_PublicPagePlaceholder_PageUserControl_ctl00_PublicPageLoadFixPage_UserControlShowEnergyAndPower1__datePicker_textBox"]').send_keys(Keys.SHIFT, Keys.ARROW_UP) 
    next4=driver.find_element_by_xpath('//*[@id="ctl00_ContentPlaceHolder1_PublicPagePlaceholder_PageUserControl_ctl00_PublicPageLoadFixPage_UserControlShowEnergyAndPower1__datePicker_textBox"]').send_keys(Keys.DELETE)
    ActionChains(driver).click(next4).perform() 
    #zadavanje vrijednosti kalendaru
    vvx=WebDriverWait(driver,60).until(EC.presence_of_all_elements_located((By.XPATH,'//*[@id="ctl00_ContentPlaceHolder1_PublicPagePlaceholder_PageUserControl_ctl00_PublicPageLoadFixPage_UserControlShowEnergyAndPower1__datePicker_textBox"]')))
    vvx=WebDriverWait(driver,60).until(EC.element_to_be_clickable((By.XPATH,'//*[@id="ctl00_ContentPlaceHolder1_PublicPagePlaceholder_PageUserControl_ctl00_PublicPageLoadFixPage_UserControlShowEnergyAndPower1__datePicker_textBox"]')))
    next5=driver.find_element_by_xpath('//*[@id="ctl00_ContentPlaceHolder1_PublicPagePlaceholder_PageUserControl_ctl00_PublicPageLoadFixPage_UserControlShowEnergyAndPower1__datePicker_textBox"]').send_keys(value)
    time.sleep(5)
    #lociranje buttona
    WebDriverWait(driver,60).until(EC.presence_of_all_elements_located((By.XPATH,'//*[@id="ctl00_ContentPlaceHolder1_PublicPagePlaceholder_PageUserControl_ctl00_PublicPageLoadFixPage_UserControlShowEnergyAndPower1__datePicker_textBox"]')))
    time.sleep(5)
    WebDriverWait(driver,60).until(EC.element_to_be_clickable((By.XPATH,'//*[@id="ctl00_ContentPlaceHolder1_PublicPagePlaceholder_PageUserControl_ctl00_PublicPageLoadFixPage_UserControlShowEnergyAndPower1__datePicker_textBox"]')))
    ActionChains(driver).click(next5).perform() 
    time.sleep(4)
    vvn1 = WebDriverWait(driver, 60).until(EC.presence_of_all_elements_located((By.ID, "ctl00_ContentPlaceHolder1_PublicPagePlaceholder_PageUserControl_ctl00_PublicPageLoadFixPage_UserControlShowEnergyAndPower1_OpenButtonsDivImg")))
    vvn1 = WebDriverWait(driver, 60).until(EC.element_to_be_clickable((By.ID, "ctl00_ContentPlaceHolder1_PublicPagePlaceholder_PageUserControl_ctl00_PublicPageLoadFixPage_UserControlShowEnergyAndPower1_OpenButtonsDivImg")))
    element_to_hover_over = driver.find_element_by_id("ctl00_ContentPlaceHolder1_PublicPagePlaceholder_PageUserControl_ctl00_PublicPageLoadFixPage_UserControlShowEnergyAndPower1_OpenButtonsDivImg")
    hover = ActionChains(driver).move_to_element(element_to_hover_over)
    hover.perform()
    
    vv = WebDriverWait(driver, 60).until(EC.presence_of_all_elements_located((By.ID, "ctl00_ContentPlaceHolder1_PublicPagePlaceholder_PageUserControl_ctl00_PublicPageLoadFixPage_UserControlShowEnergyAndPower1_ImageButtonValues")))
    #klik na button "details"
    vv=WebDriverWait(driver,60).until(EC.element_to_be_clickable((By.ID,"ctl00_ContentPlaceHolder1_PublicPagePlaceholder_PageUserControl_ctl00_PublicPageLoadFixPage_UserControlShowEnergyAndPower1_ImageButtonValues")))
    element = driver.find_element_by_id("ctl00_ContentPlaceHolder1_PublicPagePlaceholder_PageUserControl_ctl00_PublicPageLoadFixPage_UserControlShowEnergyAndPower1_ImageButtonValues")
    driver.execute_script("arguments[0].click();", element)
 
    
    #pređi na pop-up window
    window_after = driver.window_handles[1]
    driver.switch_to.window(window_after)
    #lociranje tabele
    WebDriverWait(driver,60).until(EC.presence_of_all_elements_located((By.ID,"ctl00_ContentPlaceHolder1_UserControlChartValues1_Table1")))
    table1= driver.find_element(By.ID, "ctl00_ContentPlaceHolder1_UserControlChartValues1_Table1")
    html = table1.get_attribute('innerHTML')
    soup = BeautifulSoup(html)

    
    
    dfs[k]=pd.read_html(driver.find_element(By.TAG_NAME, 'table').get_attribute('outerHTML'))[0]
    dfs[k]=dfs[k].assign(Date=date_list1[k])

  
    time.sleep(3)
    
    driver.switch_to.window(driver.window_handles[0])

In [ ]:
dfs[-1]

In [ ]:
df_tot=pd.concat(dfs,axis=0)
df_tot.to_excel("SE_2019_0211_291_2020.xlsx")